<a href="https://colab.research.google.com/github/Keenandrea/robo-trigger/blob/master/robotrigger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import pickle
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from music21 import converter, instrument, note, chord

Using TensorFlow backend.


In [0]:
run_on_colab = True
if(run_on_colab):
  from google.colab import drive
  # This will prompt for authorization.
  drive.mount('/content/drive')

In [0]:
!pip install music21;

In [16]:
from google.colab import files
files.upload()

Saving themes.zip to themes.zip


{'themes.zip': b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xd1\x95fN|\xc6w"\x9b\x02\x00\x00P\x19\x00\x00\x1d\x00\x00\x00themes/piano-ayla-s-theme.mid\xedXKk\x13Q\x18\xfd4c\'>@7\xc5A7\xd3\x85\xd8U\xedB("\x964%v\x1c&\xf3h\xb2\xd0]&\xcd\x98\x8eM2\x92\x07\xd8\x85\xd8\x99\xa1u%4\t\xfe\r\xf1\xb1\x13\x17>\xfe\x88?A\xf0\x07\xc4s\x99\xbc\x9c\xa8\x05I\xa0\x81\xbb:\xe7~\xdf\xb9\xe7\x9e{a.7\xc9\xe6wKD\xb4@g(I?\xb2\xf9\xfa\x1eF\xb7\xa9\xf7P\x10\xceJI\xeaY\x89sw\x1fPo\xe1\xd2\xc6~\xc5\xbe\xd9\x90\xf3\xbbN\xd5\x11\xa3\xf2\xcb{@\xa1\xfb=|\x95\xee\xdd\xa2hn\xf2\x02\xf5\x12b\xbe\xde\xaa>u\x9a\xf4\x99\xe8\xad\xe8\x89\xc7\x8a\x99?P\xc8/\xf7\xc9b\x0c\xd1P\xcd\xed\x03\x95\x02\x01\xa4\x00"\xc50H\x1ek \xda\x88H1DC\x07\xd1GD\x9a@\x03h\x8c\xa1\x02T\xd8\x84\x88H1Dc\xb8:\x8f\xc5c\xf1X<\x16\x8f\xc5c\xf1X\'\xc6\xf2\xcf\x0f_\x04\x89\x9fx\x11\\\xce5\xebn\xad,gj\r\xa7Z\xac8\xf4\x85\xe8\x9dX\x11\xdb\x9aIm\xc5<\x94}\x8d\xc8\xc7K`\xb9\xad\xa3\xa2\x9a\x875_GE%?\xd56P\xd1\xcc#\xf2\rT4V\xb1P\xd1Y\xc5BEg\x15e\xa0Q\x06\x1au\xa0Q\x07

In [17]:
!unzip themes.zip;

Archive:  themes.zip
  inflating: themes/piano-ayla-s-theme.mid  
  inflating: themes/piano-cathedral-one.mid  
  inflating: themes/piano-chrono-trigger.mid  
  inflating: themes/piano-end-of-time.mid  
  inflating: themes/piano-epilogue.mid  
  inflating: themes/piano-factory.mid  
  inflating: themes/piano-frog-s-theme.mid  
  inflating: themes/piano-gato-s-song.mid  
  inflating: themes/piano-guardia-castle.mid  
  inflating: themes/piano-guardia-forest.mid  
  inflating: themes/piano-lavos-theme.mid  
  inflating: themes/piano-magnus.mid  
  inflating: themes/piano-robo-s-theme.mid  
  inflating: themes/piano-schala-s-theme.mid  
  inflating: themes/piano-to-far-away-times.mid  


In [18]:
notes = []
for i,file in enumerate(glob.glob("themes/*.mid")):
  midi = converter.parse(file)
  print('\r', 'Parsing file ', i, " ",file, end='')
  notes_to_parse = None
  try: # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes
  for element in notes_to_parse:
    if isinstance(element, note.Note):
      notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
      notes.append('.'.join(str(n) for n in element.normalOrder))
with open('notes', 'wb') as filepath:
  pickle.dump(notes, filepath)

 Parsing file  14   themes/piano-to-far-away-times.mid

In [19]:
# Count different possible outputs
n_vocab = (len(set(notes)))
n_vocab

146

In [0]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length] # Size sequence_length
  sequence_out = notes[i + sequence_length]  # Size 1
  # Map pitches of sequence_in to integers
  network_input.append([note_to_int[char] for char in sequence_in])
  # Map integer of sequence_out to an integer
  network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [21]:
network_input.shape

(12999, 100, 1)

In [0]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

In [23]:
model = create_network(network_input,n_vocab)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)      

In [0]:
# In case we want to use previously trained weights
weights = ""
if(len(weights)>0): model.load_weights(weights)

In [0]:
filepath = "/content/drive/My Drive/{epoch:02d}-{loss:.4f}.h5"

checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=0,
                             save_best_only=True,mode='min')

callbacks_list = [checkpoint]
model.fit(network_input, network_output, epochs=75, batch_size=64, 
          callbacks=callbacks_list)